In [1]:
import numpy as np
import torch
from torch.utils import data
from torch import nn


In [2]:
def synthetic_data(w, b, num_examples):
    """生成 y = Xw+b 噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [3]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size=batch_size, shuffle=is_train)

In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [6]:
X, y = next(iter(data_iter))
X.shape, y.shape

(torch.Size([10, 2]), torch.Size([10, 1]))

In [7]:
net = nn.Sequential(nn.Linear(2,1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [8]:
loss = nn.MSELoss()

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch+1}, loss {l:f}')

torch.Size([10, 1]) torch.Size([10, 1])
epoch 1, loss 32.214756
torch.Size([10, 1]) torch.Size([10, 1])
epoch 2, loss 32.214756
torch.Size([10, 1]) torch.Size([10, 1])
epoch 3, loss 32.214756


In [11]:
w = net[0].weight.data
b = net[0].bias.data
print('w的估计误差:', true_w - w.reshape(true_w.shape))
print('b的估计误差:', true_b - b)

w的估计误差: tensor([ 2.0035, -3.3946])
b的估计误差: tensor([4.2000])
